In [6]:
!pip install dash==0.29.0

     |████████████████████████████████| 95 kB 6.1 MB/s  eta 0:00:01
     |████████████████████████████████| 1.0 MB 40.8 MB/s eta 0:00:01
     |████████████████████████████████| 357 kB 39.4 MB/s eta 0:00:01
  Created wheel for dash: filename=dash-0.29.0-py3-none-any.whl size=30405 sha256=5ed73637ba54d2b8f61d34b0d64f7f8a3fbcad6126918eca0e662852fbacf395
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/16/91/54/1296b6d280c77eb28ff930d92ec78fd4332ca9af2917cca72f
  Created wheel for dash-renderer: filename=dash_renderer-1.9.1-py3-none-any.whl size=1014873 sha256=d6a2483d3d2243763323dac471495ab7e8394c5f24fe69ac9f7e198d1cc66c5e
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/52/2f/28/956ce9f7763c886d23f0082829bd726e5dacd6c708a5841c77
Successfully built dash dash-renderer


In [7]:
conda install dash dash-core-components dash-html-components dash-renderer -c conda-forge

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.9

  added / updated specs:
    - dash
    - dash-core-components
    - dash-html-components
    - dash-renderer


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    brotli-python-1.0.9        |   py39he80948d_5         352 KB  conda-forge
    certifi-2021.10.8          |   py39hf3d152e_2         145 KB  conda-forge
    dash-2.3.1                 |     pyhd8ed1ab_0         7.5 MB  conda-forge
    dash-core-components-2.0.0 |     pyhd8ed1ab_1           7 KB  conda-forge
    dash-html-components-2.0.0 |     pyhd8ed1ab_1           8 KB  conda-forge
    dash-renderer-1.9.1        |     pyhd8ed1ab_0         807 KB  conda-forge
    flask-2.1.1                |     pyhd8ed1ab_0          70 KB  conda-forge
    flask-compress-1.11        |     pyhd8ed1ab_0          14 KB  conda-forge
    itsdangerous-2.1.

In [27]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_e6f728ac279840dea821db16a6a480a9 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='-Z_CUpDg9iehHgBJmSJSBi-DfmF3htFfRUClwe0qZ1Is',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

body = client_e6f728ac279840dea821db16a6a480a9.get_object(Bucket='completetheedawithvisualizationla-donotdelete-pr-bm35fowbyrivnf',Key='Spacex.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()


,Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing _Outcome
0,04-06-2010,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
1,08-12-2010,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of...",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2,22-05-2012,07:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
3,08-10-2012,00:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
4,01-03-2013,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


In [31]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

In [33]:
# Read the airline data into pandas dataframe
max_payload = df_data_1['PAYLOAD_MASS__KG_'].max()
min_payload = df_data_1['PAYLOAD_MASS__KG_'].min()

In [34]:
# Create a dash application
app = dash.Dash(__name__)

In [36]:
# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id='site-dropdown',
                                            options=[
                                                         {'label': 'ALL SITES', 'value': 'ALL'},
                                                         {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                         {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                         {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                         {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
                                                    ],
                                            value='ALL',
                                            placeholder="Select a Launch Site here", 
                                            searchable=True),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min=0,max=10000,step=1000,
                                                value=[min_payload,max_payload],
                                                marks={0: '0', 2500:'2500',5000:'5000',
                                                7500:'7500', 10000: '10000'}),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

In [37]:
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value'))

def build_graph(site_dropdown):
    if site_dropdown == 'ALL':
        piechart = px.pie(data_frame = df_data_1, names='Launch Site', values='class' ,title='Total Launches for All Sites')
        return piechart
    else:
        #specific_df = spacex_df['Launch Site']
        specific_df=df_data_1.loc[spacex_df['Launch Site'] == site_dropdown]
        piechart = px.pie(data_frame = specific_df, names='class',title='Total Launch for a Specific Site')
        return piechart

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'),
    Input(component_id='payload-slider', component_property='value')])

def update_graph(site_dropdown, payload_slider):
    if site_dropdown == 'ALL':
        filtered_data = df_data_1[(df_data_1['PAYLOAD_MASS__KG_']>=payload_slider[0])
        &(spacex_df['PAYLOAD_MASS__KG_']<=payload_slider[1])]
        scatterplot = px.scatter(data_frame=filtered_data, x="PAYLOAD_MASS__KG_", y="class", 
        color="Booster Version Category")
        return scatterplot
    else:
        specific_df=df_data_1.loc[spacex_df['Launch Site'] == site_dropdown]
        filtered_data = specific_df[(specific_df['PAYLOAD_MASS__KG_']>=payload_slider[0])
        &(df_data_1['PAYLOAD_MASS__KG_']<=payload_slider[1])]
        scatterplot = px.scatter(data_frame=filtered_data, x="PAYLOAD_MASS__KG_", y="class", 
        color="Booster Version Category")
        return scatterplot

# Run the app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
